# Milk Delivery for Hawker Centers in Singapore

Jack is a driver of ABC Milk Company. He is assigned to deliver fresh milk for 20 hawker centers each morning.  Suppose he can visit each center only once, could you help Jack to design a rounte with least amount of time (distance)?

The geolocation data of the hawker centers are imported as follows:

In [1]:
import pandas as pd
import numpy as np
import mpu
import pickle
import folium
import random
from folium.plugins import BeautifyIcon

import openrouteservice as ors
from rsome import ro
from rsome import grb_solver as grb

In [2]:
## Import the geolocation data of all the hawker centres
data = pd.read_csv('hawker_centres_kml.csv')
# print(data.head(5))

In [3]:
lon = data['X']
lat = data['Y']

N = 20
lon = [lon[i] for i in range(N)]
lat = [lat[i] for i in range(N)]


## plot the locations in singapore map
# https://livecodestream.dev/post/how-to-plot-your-data-on-maps-using-python-and-folium/
SGmap = folium.Map(location=[lat[2], lon[2]], tiles="Stamen Terrain", zoom_start=11) #Stamen Terrain, OpenStreetMap
for i in range(N):
    folium.Marker(
        location=[lat[i], lon[i]],
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(i + 1),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(SGmap)
SGmap

You searched online and realized that this exactly the traveling salesman problem, which can be solved by using the MTZ formulation. 

**(Miller–Tucker–Zemlin formulation)** Label the centres with the numbers $1,\ldots,N$ and define:
$$
x_{ij} = \begin{cases} 
1 & \text{the path goes from centre } i \text{ to centre } j \\ 
0 & \text{otherwise} 
\end{cases}
$$

For $i=1,\ldots,N$, let $u_i$ be a dummy variable, and finally take $d_{ij} > 0$ to be the distance from centre $i$ to centre $j$. Then TSP can be written as the following integer linear programming problem:
$$
\begin{align}
\min &\sum_{i=1}^n \sum_{j=1}^n d_{ij}x_{ij}\colon &&  \\
     & \sum_{i=1}^N x_{ij} = 1 && j=1, \ldots, N; \\
     & \sum_{j=1}^N x_{ij} = 1 && i=1, \ldots, N; \\
     & u_i-u_j +Nx_{ij} \le N-1 && 2 \le i \ne j \le N;  \\
     & 1 \le u_i \le N-1 && 2 \le i \le N; \\
     & u_{i} \in \mathbf{Z} && i=2, \ldots, N; \\
     & x_{ij} \in \{0,1\}  && i,j=1, \ldots, N; \\
     & x_{ii} = 0  && i= 1, \ldots, N; \\
\end{align}
$$

With the coding experience from Prescriptive Analytics, you believe that you can solve this problem using RSOME.

Tips: 
1. Distance matrix can be approximately obtained via the `haversine_distance` method of `mpu` package
2. You can save your results directly to "pickle" files using `pickle` package
3. After obtaining the optimal solution "x_star", you can use `argwhere` function to find the route
4. You can plot the route on the map by using `folium.PolyLine` method

### Some issues with the solution:

1. Distance is an approximation;

2. Routes are not on the real roads.

You can use `openrouteservice` to estimate the distance matrix and also use this package to draw the real routes for the example.

To use `openrouteservice`, you need to sign up in https://openrouteservice.org/dev/#/signup and request an API Key. 

With the real distance and solution, you can show the route on the map by using the `client.directions` method

### Extend it to a vehicle routing problem (optional)

**Due to vehicle capacity constraints, Jack can only deliver milk to some of the centers each time. To put it another way, Jack must deliver at least two times. Could you assist Jack in planning the two routes for each day? Assume that Jack can only visit each center once.**

Let $\mathcal{V} = \{0, 1,2,\ldots, N\}$. The formulation of the TSP can be extended to create the two index vehicle flow formulations for the VRP

$$ 
\begin{align}
\min & \sum_{i\in \mathcal{V}}\sum_{j \in \mathcal{V}}c_{ij}x_{ij}\\
\mbox{s.t.} & \sum_{i \in \mathcal{V}}x_{ij}=1 \quad && \forall j \in \mathcal{V}\backslash \left \{ 0 \right \} \\
&\sum_{j \in \mathcal{V}}x_{ij}=1 \quad && \forall i \in \mathcal{V}\backslash \left \{ 0 \right \}\\
&\sum_{i \in \mathcal{V}}x_{i0}=K&& \\
&\sum_{j \in \mathcal{V}}x_{0j}=K&& \\
&u_j-u_i\geq d_j-C(1-x_{ij}) && \forall i,j \in V\backslash\{0\}, i\neq j~~~~\text{s.t. } d_i +d_j \leq C\\
&0 \leq u_i \leq C-d_i && \forall i \in V\backslash \{0\}\\
&x_{ij}\in \{0,1\} \quad && \forall i,j \in \mathcal{V}\\
&x_{ii} = 0 && \forall i \in \mathcal{V}
\end{align}
$$

In this formulation $c_{ij}$ represents the cost of going from node $i$ to node $j$, $x_{ij}$ is a binary variable that has value $1$ if the arc going from $i$ to $j$ is considered as part of the solution and $0$ otherwise, $K$ is the number of available vehicles. We are also assuming that $0$ is the depot node.

The first and second constraints state that exactly one arc enters and exactly one leaves each vertex associated with a customer, respectively. The third and fourth  say that the number of vehicles leaving the depot is the same as the number entering. The fifth and sixth constraints are the capacity cut constraints, which impose that the routes must be connected and that the demand on each route must not exceed the vehicle capacity. Finally, the sixth constraints are the integrality constraints and the last constraints ensure that no arc for each node.

The fifth and sixth constraints are known as the MTZ constraints, they were first proposed for the TSP and subsequently extended by Christofides, Mingozzi and Toth
$$
u_j-u_i\geq d_j-C(1-x_{ij}) ~~~~~~\forall i,j \in V\backslash\{0\}, i\neq j~~~~\text{s.t. } d_i +d_j \leq C
$$
and
$$
0 \leq u_i \leq C-d_i ~~~~~~\forall i \in V\backslash \{0\}
$$
where $ u_i,~i \in V \backslash \{0\} $ is an additional continuous variable which represents the load left in the vehicle '''after''' visiting customer $i$ and $d_i$ is the demand of customer $i$. These impose both the connectivity and the capacity requirements. When $x_{ij}=0$ constraint then $i$ is not binding' since $u_i\leq C$ and $u_j\geq d_j$ whereas $x_{ij} = 1$ they impose that $u_j \geq u_i +d_j$.
